In [ ]:
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Jerryblessed/fraudShieldgraphai/blob/main/fdetect.ipynb)

# Install necessary libraries
!pip install networkx nx_arangodb cugraph-cu12 cudf-cu12 openai langchain_community python-telegram-bot matplotlib langgraph langchain_openai openai  nx-arangodb[llm]

In [ ]:
import os
import networkx as nx
import nx_arangodb as nxadb
import cudf
import cugraph as cg
import openai
from openai import AzureOpenAI
import matplotlib.pyplot as plt
from networkx.algorithms import community
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from telegram import Bot  # Telegram Bot API

# Load secure environment variables
os.environ["DATABASE_HOST"] = "https://tutorials.arangodb.cloud:8529"
os.environ["DATABASE_USERNAME"] = "TUThqy2rf7vstghfr8wuhbc59"
os.environ["DATABASE_PASSWORD"] = "TUTpj1exghn8jutfpibxca2g"
os.environ["DATABASE_NAME"] = "TUT0pwg4agundai61ujmyjdu"

api_base = "https://thisisoajo.openai.azure.com/"
api_key = "9I4UEJweVUdih04Uv8AXcAxs5H8jSQRfwaugcSQYHcI882wSpFvqJQQJ99BAACL93NaXJ3w3AAABACOGkv4f"
api_version = "2023-06-01-preview"

TELEGRAM_BOT_TOKEN = "your_telegram_bot_token"
TELEGRAM_CHAT_ID = "your_chat_id"



In [ ]:
# Initialize Telegram Bot
bot = Bot(token=TELEGRAM_BOT_TOKEN)

def send_telegram_message(message):
    """Send a message to Telegram chat."""
    try:
        bot.send_message(chat_id=TELEGRAM_CHAT_ID, text=message)
    except Exception as e:
        print(f"Telegram Error: {e}")




In [ ]:
# Graph Analysis Functions
def initialize_graph():
    """Load a sample graph and persist it to ArangoDB."""
    G_nx = nx.karate_club_graph()
    G_adb = nxadb.Graph(name="GraphRAG")
    try:
        G_adb.add_edges_from([(u, v, {k: val for k, val in d.items() if k not in {'_from', '_to'}}) for u, v, d in G_nx.edges(data=True)])
        send_telegram_message("✅ Graph successfully initialized in ArangoDB.")
    except KeyError as e:
        error_msg = f"❌ Error adding edges: {e}"
        print(error_msg)
        send_telegram_message(error_msg)
    return G_adb

def analyze_graph_cugraph(G_nx):
    """Convert to cuGraph for GPU acceleration, fallback to CPU if no CUDA."""
    try:
        if cg.device_context().is_cuda:
            cu_df = cudf.DataFrame(list(G_nx.edges(data=True)), columns=["src", "dst", "weight"])
            G_cg = cg.Graph()
            G_cg.from_cudf_edgelist(cu_df, source="src", destination="dst", edge_attr="weight")
            send_telegram_message("⚡ Graph successfully analyzed using cuGraph (GPU-accelerated).")
            return G_cg
    except Exception as e:
        fallback_msg = f"⚠️ GPU not available or error using cuGraph: {e}. Falling back to NetworkX."
        print(fallback_msg)
        send_telegram_message(fallback_msg)
    return G_nx

def visualize_graph(G_adb):
    """Convert ArangoDB graph to NetworkX and visualize it."""
    G_nx = nx.Graph(G_adb)
    plt.figure(figsize=(8, 6))
    nx.draw(G_nx, with_labels=True, node_color='lightblue', edge_color='gray')
    plt.title("Graph Visualization")
    plt.show()
    send_telegram_message("📊 Graph visualization displayed.")



In [ ]:
# OpenAI Chat Integration
llm = ChatOpenAI(model_name="gpt-4o", openai_api_key=api_key, openai_api_base=api_base)

def query_graph(prompt):
    """Query the graph using GPT-4o and LangChain."""
    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an AI assistant specializing in graph analytics."),
        ("user", "{prompt}")
    ])
    response = llm.invoke(chat_prompt.format_messages(prompt=prompt))
    send_telegram_message(f"📡 User Query: {prompt}\n🤖 AI Response: {response}")
    return response



In [ ]:
# Run in Jupyter Notebook
G_adb = initialize_graph()
visualize_graph(G_adb)

while True:
    user_query = input("Ask a graph-related question: ")
    if user_query.lower() in ["exit", "quit"]:
        send_telegram_message("🔴 Graph interaction session ended.")
        print("Goodbye!")
        break
    response = query_graph(user_query)
    print("AI Response:", response)